In [42]:
import numpy as np
import pandas as pd
import featuretools as ft
import featuretools.variable_types as vtypes

import os
PATH = '/home/ubuntu/data/astro/partitions/'

train = pd.read_csv(f'{PATH}ptrain/test_set.csv')
train_meta = pd.read_csv(f'{PATH}ptrain/test_set_metadata.csv')

In [16]:
train.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [17]:
train_meta.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [225]:
class AstroP():
    def __init__(self, partition, train = False, feature_defs = None):
        self.partition = partition
        # Read in data
        self.data = pd.read_csv(f'{PATH}p{partition}/test_set.csv')
        self.metadata = pd.read_csv(f'{PATH}p{partition}/test_set_metadata.csv')
        
        # If training, extract the target and run deep feature synthesis
        if train:
            self.target = np.array(self.metadata.pop('target'))
        
        # Make an entityset
        self.es = self.make_entityset()
        
        if not train:
            assert(feature_defs is not None), "Feature defs must be provided with testing set"
            
        self.feature_defs = feature_defs
        
        # If features have already been calculated for this partition
        if os.path.exists(f'{PATH}p{partition}/fm.csv'):
            self.existing_feature_matrix = pd.read_csv(f'{PATH}p{partition}/fm.csv', nrows = 1)
            if "Unnamed: 0" in self.existing_feature_matrix:
                self.existing_feature_matrix.drop(columns  = ["Unnamed: 0"], inplace = True)
            self.feature_matrix = self.existing_feature_matrix.copy()
            self.calculated_features = list(self.feature_matrix.columns)
        else:
            self.calculated_features = []
            self.existing_feature_matrix = None
            self.feature_matrix = None
            
        if os.path.exists(f'{PATH}p{partition}/features.txt'):
            self.calculated_feature_defs = ft.load_features(f'{PATH}p{partition}/features.txt')
        else:
            self.calculated_feature_defs = []

        
    def make_entityset(self):
        es = ft.EntitySet(id = 'astro')
        es.entity_from_dataframe(entity_id = 'data', dataframe = self.data,
                                 make_index = True, index = 'index', time_index = 'mjd', 
                                 variable_types = {'detected': vtypes.Categorical,
                                                   'passband': vtypes.Categorical})
        es.entity_from_dataframe(entity_id = 'objects', dataframe = self.metadata,
                                 index = 'object_id',
                                 variable_types = {'ddf': vtypes.Categorical})
        
        es.add_relationship(ft.Relationship(es['objects']['object_id'], es['data']['object_id']))
        return es

    def deep_feature_synthesis(self, features_only = True):
        if features_only:
            self.feature_defs = ft.dfs(entityset = self.es, 
                                       agg_primitives = ['last', 'min', 'max', 'count', 'sum', 'std'],
                                       trans_primitives = ['time_since_previous', 'cum_max', 'cum_sum', 'cum_mean'],
                                       target_entity = 'objects', 
                                       features_only = features_only)
        else:
            self.feature_matrix, self.feature_defs = ft.dfs(entityset = self.es,
                                                            agg_primitives = ['last', 'min', 'max', 'count', 'sum', 'std'],
                                                            trans_primitives = ['time_since_previous', 'cum_max', 'cum_sum', 'cum_mean'],
                                                            target_entity = 'objects', 
                                                            features_only = features_only)
            
    def calculate_features(self):
        assert(self.feature_defs is not None), "Feature definitions must be provided"
        # Remove the features that have already been calculated
        self.feature_defs = [f for f in self.feature_defs if f.get_name() not in self.calculated_features]
        
        if self.feature_defs != []:
        
            # Calcuate the feature matrix
            self.feature_matrix = ft.calculate_feature_matrix(self.feature_defs, entityset = self.es)
            
            if self.existing_feature_matrix is not None:
                self.existing_feature_matrix = self.existing_feature_matrix[[c for c in self.existing_feature_matrix if c not in self.feature_matrix]]
                self.feature_matrix = pd.concat([self.feature_matrix, self.existing_feature_matrix], axis = 1)
            
            # Add the features to those already calculated for this feature matrix
            self.calculated_features = list(self.feature_matrix.columns)
            self.calculated_feature_defs.extend(self.feature_defs)
            
        else:
            print('All Features Already Calculated for Partition')
            self.feature_matrix = self.existing_feature_matrix.copy()
    
    def save_features(self):
        assert(self.feature_matrix is not None), "Feature matrix must be calculated"
                
        self.feature_matrix.to_csv(f'{PATH}p{self.partition}/fm.csv')
        ft.save_features(self.calculated_feature_defs, f'{PATH}p{self.partition}/features.txt')
        
        
    def make_predictions(self, trained_model, fit_imputer):            
        self.train_features = fit_imputer.transform(self.feature_matrix.drop(columns = ['object_id']))
        self.predictions = trained_model.predict_proba(self.train_features)
        
        self.prediction_df = pd.DataFrame(columns = ['object_id',
                                                     'class_6',
                                                     'class_15',
                                                     'class_16',
                                                     'class_42',
                                                     'class_52',
                                                     'class_53',
                                                     'class_62',
                                                     'class_64',
                                                     'class_65',
                                                     'class_67',
                                                     'class_88',
                                                     'class_90',
                                                     'class_92',
                                                     'class_95',
                                                     'class_99'])
        
        self.prediction_df['object_id'] = self.feature_matrix['object_id']
        self.prediction_df.loc[:, 'class_6': 'class_95'] = self.predictions
        self.prediction_df['class_99'] = 0
        
        self.prediction_df.to_csv(f'{PATH}p{self.partition}/predictions.csv', index = False)

In [226]:
sample_sub = pd.read_csv('/home/ubuntu/data/astro/sample_submission.csv', nrows = 1)
list(sample_sub.columns)

['object_id',
 'class_6',
 'class_15',
 'class_16',
 'class_42',
 'class_52',
 'class_53',
 'class_62',
 'class_64',
 'class_65',
 'class_67',
 'class_88',
 'class_90',
 'class_92',
 'class_95',
 'class_99']

In [227]:
trainp = AstroP('train', train = True)
trainp.deep_feature_synthesis()

In [228]:
trainp.calculated_features

['object_id',
 'ra',
 'decl',
 'gal_l',
 'gal_b',
 'hostgal_specz',
 'hostgal_photoz',
 'hostgal_photoz_err',
 'distmod',
 'mwebv',
 'target',
 'ddf',
 'LAST(data.mjd)',
 'LAST(data.flux)',
 'LAST(data.flux_err)',
 'LAST(data.detected)',
 'LAST(data.passband)',
 'MIN(data.mjd)',
 'MIN(data.flux)',
 'MIN(data.flux_err)',
 'MAX(data.mjd)',
 'MAX(data.flux)',
 'MAX(data.flux_err)',
 'COUNT(data)',
 'SUM(data.mjd)',
 'SUM(data.flux)',
 'SUM(data.flux_err)',
 'STD(data.mjd)',
 'STD(data.flux)',
 'STD(data.flux_err)',
 'LAST(data.CUM_MAX(flux by object_id))',
 'LAST(data.CUM_MAX(flux_err by object_id))',
 'LAST(data.CUM_SUM(flux by object_id))',
 'LAST(data.CUM_SUM(flux_err by object_id))',
 'LAST(data.CUM_MEAN(flux by object_id))',
 'LAST(data.CUM_MEAN(flux_err by object_id))',
 'MIN(data.CUM_MAX(flux by object_id))',
 'MIN(data.CUM_MAX(flux_err by object_id))',
 'MIN(data.CUM_SUM(flux by object_id))',
 'MIN(data.CUM_SUM(flux_err by object_id))',
 'MIN(data.CUM_MEAN(flux by object_id))',
 '

In [229]:
trainp.calculate_features()

All Features Already Calculated for Partition


In [230]:
features = ft.load_features(f'{PATH}ptrain/features.txt')
features[0].get_name()

'ra'

In [231]:
feature_matrix = pd.read_csv(f'{PATH}ptrain/fm.csv')
feature_matrix.head()

,object_id,ra,decl,gal_l,gal_b,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,ddf,LAST(data.mjd),LAST(data.flux),LAST(data.flux_err),LAST(data.detected),LAST(data.passband),MIN(data.mjd),MIN(data.flux),MIN(data.flux_err),MAX(data.mjd),MAX(data.flux),MAX(data.flux_err),COUNT(data),SUM(data.mjd),SUM(data.flux),SUM(data.flux_err),STD(data.mjd),STD(data.flux),STD(data.flux_err),LAST(data.CUM_MAX(flux by object_id)),LAST(data.CUM_MAX(flux_err by object_id)),LAST(data.CUM_SUM(flux by object_id)),LAST(data.CUM_SUM(flux_err by object_id)),LAST(data.CUM_MEAN(flux by object_id)),LAST(data.CUM_MEAN(flux_err by object_id)),MIN(data.CUM_MAX(flux by object_id)),MIN(data.CUM_MAX(flux_err by object_id)),MIN(data.CUM_SUM(flux by object_id)),MIN(data.CUM_SUM(flux_err by object_id)),MIN(data.CUM_MEAN(flux by object_id)),MIN(data.CUM_MEAN(flux_err by object_id)),MAX(data.CUM_MAX(flux by object_id)),MAX(data.CUM_MAX(flux_err by object_id)),MAX(data.CUM_SUM(flux by object_id)),MAX(data.CUM_SUM(flux_err by object_id)),MAX(data.CUM_MEAN(flux by object_id)),MAX(data.CUM_MEAN(flux_err by object_id)),SUM(data.CUM_MAX(flux by object_id)),SUM(data.CUM_MAX(flux_err by object_id)),SUM(data.CUM_SUM(flux by object_id)),SUM(data.CUM_SUM(flux_err by object_id)),SUM(data.CUM_MEAN(flux by object_id)),SUM(data.CUM_MEAN(flux_err by object_id)),STD(data.CUM_MAX(flux by object_id)),STD(data.CUM_MAX(flux_err by object_id)),STD(data.CUM_SUM(flux by object_id)),STD(data.CUM_SUM(flux_err by object_id)),STD(data.CUM_MEAN(flux by object_id)),STD(data.CUM_MEAN(flux_err by object_id))
0,615,349.046051,-61.943836,320.796530,-51.753706,0.0000,0.0000,0.0000,NaN,0.017,92,1,60624.2132,157.080200,8.453112,1,5,59750.4229,-1100.440063,2.130510,60624.2132,660.626343,12.845472,352,2.118879e+07,-43330.143249,1577.925390,310.024487,394.109851,1.744747,660.626343,12.845472,-43330.143249,1577.925390,-123.096998,4.482743,-544.810303,3.622952,-47609.703104,3.622952,-680.622314,3.622952,660.626343,12.845472,-544.810303,1577.925390,-123.096998,6.093141,188975.718945,4255.575773,-1.156206e+07,281899.617014,-86522.016147,1625.083610,286.469679,0.986632,11859.311520,456.138608,122.080641,0.243272
1,713,53.085938,-27.784405,223.525509,-54.460748,1.8181,1.6267,0.2552,45.4063,0.007,88,1,60674.0798,-8.669188,2.216094,0,0,59825.2600,-14.735178,0.639458,60674.0798,14.770886,9.115748,350,2.108987e+07,-498.172760,825.866990,310.247414,6.471144,1.509888,14.770886,9.115748,-498.172760,825.866990,-1.423351,2.359620,9.110147,1.013889,-498.172760,1.013889,-1.423351,1.013889,14.770886,9.115748,657.694251,825.866990,9.110147,2.857584,4936.586924,3038.174287,9.574745e+04,145632.756253,981.928803,826.137140,1.187667,1.059203,314.762389,240.883371,2.295335,0.131081
2,730,33.574219,-6.579593,170.455585,-61.548219,0.2320,0.2262,0.0157,40.2561,0.021,42,1,60652.1660,-1.497761,5.210212,0,5,59798.3205,-19.159811,0.695106,60652.1660,47.310059,11.281384,330,1.987324e+07,748.253237,815.450286,302.781791,8.022239,1.721134,47.310059,11.281384,748.253237,815.450286,2.267434,2.471061,1.177371,1.364300,-21.964732,1.364300,-1.178117,1.261774,47.310059,11.281384,752.289347,815.450286,2.423176,2.852247,7373.074188,2999.147222,6.530834e+04,136359.215481,266.444608,817.541390,17.189027,2.247293,286.501217,241.159072,0.964939,0.134242
3,745,0.189873,-45.586655,328.254458,-68.969298,0.3037,0.2813,1.1523,40.7951,0.007,90,1,60624.0722,2.976642,4.417903,0,5,59770.3662,-15.494463,0.567170,60624.0722,220.795212,55.892746,351,2.113014e+07,3127.131254,897.007002,306.144145,27.558208,3.537324,220.795212,55.892746,3127.131254,897.007002,8.909206,2.555576,0.161398,0.619084,-5.122443,0.619084,-0.720523,0.610841,220.795212,55.892746,3145.846778,897.007002,14.895624,2.778598,51375.481456,7028.121168,6.275523e+05,151328.660460,2718.724880,843.419415,100.229520,17.822295,1417.156478,262.779735,5.851403,0.227629
4,1124,352.711273,-63.823658,316.922299,-51.059403,0.1934,0.2415,0.0176,40.4166,0.024,90,1,60624.2132,6.331686,7.4964

In [232]:
trainp.deep_feature_synthesis()
trainp.feature_defs

[<Feature: ra>,
 <Feature: decl>,
 <Feature: gal_l>,
 <Feature: gal_b>,
 <Feature: hostgal_specz>,
 <Feature: hostgal_photoz>,
 <Feature: hostgal_photoz_err>,
 <Feature: distmod>,
 <Feature: mwebv>,
 <Feature: ddf>,
 <Feature: LAST(data.mjd)>,
 <Feature: LAST(data.flux)>,
 <Feature: LAST(data.flux_err)>,
 <Feature: LAST(data.detected)>,
 <Feature: LAST(data.passband)>,
 <Feature: MIN(data.mjd)>,
 <Feature: MIN(data.flux)>,
 <Feature: MIN(data.flux_err)>,
 <Feature: MAX(data.mjd)>,
 <Feature: MAX(data.flux)>,
 <Feature: MAX(data.flux_err)>,
 <Feature: COUNT(data)>,
 <Feature: SUM(data.mjd)>,
 <Feature: SUM(data.flux)>,
 <Feature: SUM(data.flux_err)>,
 <Feature: STD(data.mjd)>,
 <Feature: STD(data.flux)>,
 <Feature: STD(data.flux_err)>,
 <Feature: LAST(data.CUM_MAX(flux by object_id))>,
 <Feature: LAST(data.CUM_MAX(flux_err by object_id))>,
 <Feature: LAST(data.CUM_SUM(flux by object_id))>,
 <Feature: LAST(data.CUM_SUM(flux_err by object_id))>,
 <Feature: LAST(data.CUM_MEAN(flux by objec

In [185]:
trainp.es

Entityset: astro
  Entities:
    data [Rows: 1421705, Columns: 7]
    objects [Rows: 7848, Columns: 11]
  Relationships:
    data.object_id -> objects.object_id

In [186]:
trainp.es['data']

Entity: data
  Variables:
    index (dtype: index)
    object_id (dtype: id)
    mjd (dtype: numeric_time_index)
    flux (dtype: numeric)
    flux_err (dtype: numeric)
    detected (dtype: categorical)
    passband (dtype: categorical)
  Shape:
    (Rows: 1421705, Columns: 7)

In [103]:
partitions = list(range(10))

for p in partitions:
    astrop = AstroP(partition=p, train = False, feature_defs = trainp.feature_defs)
    astrop.calculate_features()
    astrop.save_features()

In [233]:
train_meta.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [234]:
pd.options.display.max_columns = 100
feature_matrix.head()

,object_id,ra,decl,gal_l,gal_b,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,ddf,LAST(data.mjd),LAST(data.flux),LAST(data.flux_err),LAST(data.detected),LAST(data.passband),MIN(data.mjd),MIN(data.flux),MIN(data.flux_err),MAX(data.mjd),MAX(data.flux),MAX(data.flux_err),COUNT(data),SUM(data.mjd),SUM(data.flux),SUM(data.flux_err),STD(data.mjd),STD(data.flux),STD(data.flux_err),LAST(data.CUM_MAX(flux by object_id)),LAST(data.CUM_MAX(flux_err by object_id)),LAST(data.CUM_SUM(flux by object_id)),LAST(data.CUM_SUM(flux_err by object_id)),LAST(data.CUM_MEAN(flux by object_id)),LAST(data.CUM_MEAN(flux_err by object_id)),MIN(data.CUM_MAX(flux by object_id)),MIN(data.CUM_MAX(flux_err by object_id)),MIN(data.CUM_SUM(flux by object_id)),MIN(data.CUM_SUM(flux_err by object_id)),MIN(data.CUM_MEAN(flux by object_id)),MIN(data.CUM_MEAN(flux_err by object_id)),MAX(data.CUM_MAX(flux by object_id)),MAX(data.CUM_MAX(flux_err by object_id)),MAX(data.CUM_SUM(flux by object_id)),MAX(data.CUM_SUM(flux_err by object_id)),MAX(data.CUM_MEAN(flux by object_id)),MAX(data.CUM_MEAN(flux_err by object_id)),SUM(data.CUM_MAX(flux by object_id)),SUM(data.CUM_MAX(flux_err by object_id)),SUM(data.CUM_SUM(flux by object_id)),SUM(data.CUM_SUM(flux_err by object_id)),SUM(data.CUM_MEAN(flux by object_id)),SUM(data.CUM_MEAN(flux_err by object_id)),STD(data.CUM_MAX(flux by object_id)),STD(data.CUM_MAX(flux_err by object_id)),STD(data.CUM_SUM(flux by object_id)),STD(data.CUM_SUM(flux_err by object_id)),STD(data.CUM_MEAN(flux by object_id)),STD(data.CUM_MEAN(flux_err by object_id))
0,615,349.046051,-61.943836,320.796530,-51.753706,0.0000,0.0000,0.0000,NaN,0.017,92,1,60624.2132,157.080200,8.453112,1,5,59750.4229,-1100.440063,2.130510,60624.2132,660.626343,12.845472,352,2.118879e+07,-43330.143249,1577.925390,310.024487,394.109851,1.744747,660.626343,12.845472,-43330.143249,1577.925390,-123.096998,4.482743,-544.810303,3.622952,-47609.703104,3.622952,-680.622314,3.622952,660.626343,12.845472,-544.810303,1577.925390,-123.096998,6.093141,188975.718945,4255.575773,-1.156206e+07,281899.617014,-86522.016147,1625.083610,286.469679,0.986632,11859.311520,456.138608,122.080641,0.243272
1,713,53.085938,-27.784405,223.525509,-54.460748,1.8181,1.6267,0.2552,45.4063,0.007,88,1,60674.0798,-8.669188,2.216094,0,0,59825.2600,-14.735178,0.639458,60674.0798,14.770886,9.115748,350,2.108987e+07,-498.172760,825.866990,310.247414,6.471144,1.509888,14.770886,9.115748,-498.172760,825.866990,-1.423351,2.359620,9.110147,1.013889,-498.172760,1.013889,-1.423351,1.013889,14.770886,9.115748,657.694251,825.866990,9.110147,2.857584,4936.586924,3038.174287,9.574745e+04,145632.756253,981.928803,826.137140,1.187667,1.059203,314.762389,240.883371,2.295335,0.131081
2,730,33.574219,-6.579593,170.455585,-61.548219,0.2320,0.2262,0.0157,40.2561,0.021,42,1,60652.1660,-1.497761,5.210212,0,5,59798.3205,-19.159811,0.695106,60652.1660,47.310059,11.281384,330,1.987324e+07,748.253237,815.450286,302.781791,8.022239,1.721134,47.310059,11.281384,748.253237,815.450286,2.267434,2.471061,1.177371,1.364300,-21.964732,1.364300,-1.178117,1.261774,47.310059,11.281384,752.289347,815.450286,2.423176,2.852247,7373.074188,2999.147222,6.530834e+04,136359.215481,266.444608,817.541390,17.189027,2.247293,286.501217,241.159072,0.964939,0.134242
3,745,0.189873,-45.586655,328.254458,-68.969298,0.3037,0.2813,1.1523,40.7951,0.007,90,1,60624.0722,2.976642,4.417903,0,5,59770.3662,-15.494463,0.567170,60624.0722,220.795212,55.892746,351,2.113014e+07,3127.131254,897.007002,306.144145,27.558208,3.537324,220.795212,55.892746,3127.131254,897.007002,8.909206,2.555576,0.161398,0.619084,-5.122443,0.619084,-0.720523,0.610841,220.795212,55.892746,3145.846778,897.007002,14.895624,2.778598,51375.481456,7028.121168,6.275523e+05,151328.660460,2718.724880,843.419415,100.229520,17.822295,1417.156478,262.779735,5.851403,0.227629
4,1124,352.711273,-63.823658,316.922299,-51.059403,0.1934,0.2415,0.0176,40.4166,0.024,90,1,60624.2132,6.331686,7.4964

In [235]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
train_features = imputer.fit_transform(feature_matrix.drop(columns = ['object_id', 'target']))
model = RandomForestClassifier(n_estimators = 100, max_depth = 20)
model.fit(train_features, np.array(train_meta['target']))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [236]:
train_features.shape

(7848, 58)

In [237]:
feature_matrix.shape

(7848, 60)

In [238]:
fm.shape

(1, 60)

In [239]:
trainp.make_predictions(model, imputer)

ValueError: X has 59 features per sample, expected 58

In [ ]:
trainp.train_features.shape

In [ ]:
feature_matrix.head()

In [ ]:
fm = pd.read_csv('/home/ubuntu/data/astro/partitions/p0/fm.csv')
fm.head()

In [ ]:
for c in feature_matrix:
    if c not in fm:
        print(c)

In [240]:
s = imputer.transform

In [241]:
partitions = list(range(10))

for p in partitions:
    astrop = AstroP(partition=p, train = False, feature_defs = trainp.feature_defs)
    astrop.calculate_features()
    astrop.save_features()
    astrop.make_predictions(model, imputer)

All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition
All Features Already Calculated for Partition


In [222]:
astrop.feature_matrix.head()

,object_id,ra,decl,gal_l,gal_b,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,ddf,LAST(data.mjd),LAST(data.flux),LAST(data.flux_err),LAST(data.detected),LAST(data.passband),MIN(data.mjd),MIN(data.flux),MIN(data.flux_err),MAX(data.mjd),MAX(data.flux),MAX(data.flux_err),COUNT(data),SUM(data.mjd),SUM(data.flux),SUM(data.flux_err),STD(data.mjd),STD(data.flux),STD(data.flux_err),LAST(data.CUM_MAX(flux by object_id)),LAST(data.CUM_MAX(flux_err by object_id)),LAST(data.CUM_SUM(flux by object_id)),LAST(data.CUM_SUM(flux_err by object_id)),LAST(data.CUM_MEAN(flux by object_id)),LAST(data.CUM_MEAN(flux_err by object_id)),MIN(data.CUM_MAX(flux by object_id)),MIN(data.CUM_MAX(flux_err by object_id)),MIN(data.CUM_SUM(flux by object_id)),MIN(data.CUM_SUM(flux_err by object_id)),MIN(data.CUM_MEAN(flux by object_id)),MIN(data.CUM_MEAN(flux_err by object_id)),MAX(data.CUM_MAX(flux by object_id)),MAX(data.CUM_MAX(flux_err by object_id)),MAX(data.CUM_SUM(flux by object_id)),MAX(data.CUM_SUM(flux_err by object_id)),MAX(data.CUM_MEAN(flux by object_id)),MAX(data.CUM_MEAN(flux_err by object_id)),SUM(data.CUM_MAX(flux by object_id)),SUM(data.CUM_MAX(flux_err by object_id)),SUM(data.CUM_SUM(flux by object_id)),SUM(data.CUM_SUM(flux_err by object_id)),SUM(data.CUM_MEAN(flux by object_id)),SUM(data.CUM_MEAN(flux_err by object_id)),STD(data.CUM_MAX(flux by object_id)),STD(data.CUM_MAX(flux_err by object_id)),STD(data.CUM_SUM(flux by object_id)),STD(data.CUM_SUM(flux_err by object_id)),STD(data.CUM_MEAN(flux by object_id)),STD(data.CUM_MEAN(flux_err by object_id))
0,2000,349.046051,-61.943836,320.79653,-51.753706,NaN,0.6559,0.024,42.9743,0.017,1,60624.2132,-4.061143,7.437538,0,5,59750.4229,-10.485474,0.666862,60624.2132,21.316643,11.294422,352,2.118879e+07,466.790663,954.267909,310.024487,4.25823,1.924752,21.316643,11.294422,466.790663,954.267909,1.32611,2.710988,0.655506,1.507761,-9.787856,1.507761,-2.259973,1.507761,21.316643,11.294422,485.119027,954.267909,5.076399,4.062942,7060.415514,3786.604455,138230.513719,169807.305238,898.740757,974.688414,4.346168,0.947024,142.473534,275.703799,1.352682,0.170264


In [224]:
for c in feature_matrix:
    if c not in astrop.feature_matrix:
        print(c)

target


In [242]:
import findspark
findspark.init('/usr/local/spark-2.3.2-bin-hadoop2.7/')
import pyspark

In [260]:
spark_conf = pyspark.SparkConf()
# spark_conf.set('num.executors', 1)
spark_conf.set('spark.executor.memory', '56G')
spark_conf.getAll()

[('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.memory', '56G'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [264]:
sc.stop()
sc = pyspark.SparkContext(master = 'spark://ip-172-31-79-46.ec2.internal:7077',
                                     appName = 'astro', conf = spark_conf)

In [265]:
def partition_prediction(p):
    astrop = AstroP(partition=p, train = False, feature_defs = trainp.feature_defs)
    astrop.calculate_features()
    astrop.save_features()
    astrop.make_predictions(model, imputer)

In [ ]:
sc.parallelize(list(range(1000)), numSlices = 1000).\
   map(partition_prediction).collect()
sc.stop()